In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema('microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
#ta3p100.ProofreadLabelFinal & 'status="complete"'

In [4]:
@schema
class CoarseLabelFinal(dj.Computed):
    definition = """
    # Vertex labels for ta3p100.ProofreadLabel did not correctly match the triangle labels, so these are regenerated from the correct triangle labels.
    -> ta3p100.ProofreadLabelFinal
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as ta3p100.ProofreadLabel
    """
    
    key_source = ta3p100.ProofreadLabelFinal & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (ta3p100.ProofreadLabelFinal & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (ta3p100.CleansedMesh & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [5]:
start = time.time()
CoarseLabelFinal.populate(reserve_jobs=True)
print(time.time() - start)

648518346349473821
Segment 648518346349473821 vertex labels regenerated in: 2.821796178817749 seconds.
648518346349473830
Segment 648518346349473830 vertex labels regenerated in: 2.430651903152466 seconds.
648518346349473833
Segment 648518346349473833 vertex labels regenerated in: 2.4253056049346924 seconds.
648518346349473838
Segment 648518346349473838 vertex labels regenerated in: 3.460170269012451 seconds.
648518346349473847
Segment 648518346349473847 vertex labels regenerated in: 3.5508038997650146 seconds.
648518346349473907
Segment 648518346349473907 vertex labels regenerated in: 1.9856810569763184 seconds.
648518346349474146
Segment 648518346349474146 vertex labels regenerated in: 3.7000303268432617 seconds.
648518346349475035
Segment 648518346349475035 vertex labels regenerated in: 4.255794525146484 seconds.
648518346349475120
Segment 648518346349475120 vertex labels regenerated in: 1.9115664958953857 seconds.
648518346349475138
Segment 648518346349475138 vertex labels regenera

In [11]:
#(schema.jobs & "table_name='__coarse_label_final'").delete()